In [1]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [56]:
def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=LAPTOP-ABDP6J31\\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    return pyodbc.connect(conn_str)

conn = fetch_data_from_sql()
# Define the SQL query to fetch customer reviews data
query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    
    # Execute the query and fetch the data into a DataFrame
customer_review_df = pd.read_sql(query, conn)
conn.close()



C:\Users\abhis\AppData\Local\Temp\ipykernel_7500\300698681.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_review_df = pd.read_sql(query, conn)


In [57]:
customer_review_df
    

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [58]:
sia=SentimentIntensityAnalyzer()

In [67]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [75]:
def categorize_sentiment(score, rating):
    if score >0.05:
        if rating > 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score <-0.05:
        if rating <=2:
            return 'Negative'
        elif rating ==3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >=4:
            return 'Positive'
        elif rating <=2:
            return 'Negative'
            

In [68]:
customer_review_df['SentimentScore'] = customer_review_df['ReviewText'].apply(calculate_sentiment)

In [78]:
customer_review_df['SentimentScore'] = pd.to_numeric(customer_review_df['SentimentScore'], errors='coerce')


In [79]:
customer_review_df['SentimentCategory'] = customer_review_df.apply(
lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [81]:
customer_review_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)